In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
import finetuning_cnn
import importlib
importlib.reload(finetuning_cnn)
from finetuning_cnn import FineTuningCNN

import resnet_arcface
importlib.reload(resnet_arcface)
from resnet_arcface import ArcFaceNet
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(5),           # Piccole rotazioni ±5°
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),  # Traslazioni leggere
    transforms.ColorJitter(brightness=0.1, contrast=0.1),        # Leggera variazione luminosità/contrasto
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Transformations: Resize, Normalize
transform_test = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to fixed size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [3]:
# --- Dataset base (senza transform) ---
dataset = ImageFolder(root='../../IAM+RIMES')  # nessuna transform qui

# --- Carica lo split ---
#split = torch.load('splits/dataset_split.pth')
#train_indices = split['train_indices']
#val_indices = split['val_indices']
split = torch.load('splits/IAM+RIMES.pth')

train_indices = split['train_indices']
test_indices = split['test_indices']
label_map = split['label_map']

print(f"Numero train samples: {len(train_indices)}")
print(f"Numero test samples: {len(test_indices)}")
print(f"Numero classi (label ricodificati): {len(label_map)}")

# --- Applica lo split ---
train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, test_indices)

class TransformedSubset(torch.utils.data.Dataset):
    def __init__(self, subset, transform, label_map):
        self.subset = subset
        self.transform = transform
        self.label_map = label_map  # dizionario {label_originale: label_ricodificato}
    def __getitem__(self, idx):
        img, label = self.subset[idx]
        if label in self.label_map:
            mapped_label = self.label_map[label]
        else:
            # Label non autorizzata, assegna -1 o altra label "speciale"
            mapped_label = -1
        return self.transform(img), mapped_label

    def __len__(self):
        return len(self.subset)

# --- Applica le trasformazioni specifiche ---
train_data = TransformedSubset(train_subset, transform_train, label_map)

# --- Calcola il numero di classi a partire dal training set ---
all_labels = [label for _, label in train_subset]

num_classes = len(label_map)
print(f"Numero di classi (utenti autorizzati): {num_classes}")

val_data = TransformedSubset(val_subset, transform_test, label_map)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

Numero train samples: 550
Numero test samples: 150
Numero classi (label ricodificati): 100
Numero di classi (utenti autorizzati): 100


In [25]:
all_labels = [label for _, label in train_subset]
num_classes = len(set(all_labels))
print(num_classes)
print(min(all_labels), max(all_labels))

127
0 656


In [ ]:
def get_optimizers(model, stage="head"):
    if stage == "head":
        for param in model.backbone.parameters():
            param.requires_grad = False
        optimizer = torch.optim.AdamW([
            {"params": model.embedding_layer.parameters(), "lr": 1e-3},
            {"params": model.arcface.parameters(), "lr": 1e-3}
        ], weight_decay=5e-4)
    elif stage == "finetune":
        for param in model.backbone.parameters():
            param.requires_grad = True
        optimizer = torch.optim.AdamW([
            {"params": model.backbone.parameters(), "lr": 1e-4},
            {"params": model.embedding_layer.parameters(), "lr": 1e-3},
            {"params": model.arcface.parameters(), "lr": 1e-3}
        ], weight_decay=5e-4)
    return optimizer

# -----------------------------
# Setup modello e parametri
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ArcFaceNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
threshold = 0.6

# Esempio di calcolo probabilità stabile:
# logits, embeddings = model(images, labels)
# probs = F.softmax(logits, dim=1)  # già numericamente stabile grazie alla sottrazione del max


num_epochs_warmup = 5
lr_warmup = 5e-3  # leggermente più alto
num_epochs_finetune = 30

train_losses = []
train_accuracies = []
val_auth_accuracies = []
val_far_rates = []
val_frr_rates = []
val_overall_accuracies = []

best_overall_acc = 0.0  # per salvare il checkpoint migliore

# -----------------------------
# Warm-up epochs
# -----------------------------
optimizer = get_optimizers(model, stage="head")
for param_group in optimizer.param_groups:
    param_group['lr'] = lr_warmup

print("Inizio warm-up...")
for epoch in range(num_epochs_warmup):
    # --- TRAIN ---
    model.train()
    running_loss = correct_train = total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # prime 2 epoche: forward open-set
        logits, embeddings = model(images, labels=None if epoch < 2 else labels)
        
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct_train / total_train)

    # --- VALIDATION OPEN-SET ---
    model.eval()
    correct_auth = total_auth = 0
    false_accepts = false_rejects = 0
    total_unauth = total_test = correct_total = 0
    confidence_sum = confidence_count = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            logits, embeddings = model(images, labels=None)
            # softmax stabile
            probs = F.softmax(logits, dim=1)
            max_probs, predicted = torch.max(probs, 1)

            for pred, label, conf in zip(predicted, labels, max_probs):
                label, pred, conf = label.item(), pred.item(), conf.item()
                confidence_sum += conf
                confidence_count += 1

                if conf < threshold:
                    pred = -1

                if label == -1:
                    total_unauth += 1
                    if pred != -1:
                        false_accepts += 1
                else:
                    total_auth += 1
                    if pred == label:
                        correct_auth += 1
                    else:
                        false_rejects += 1

                if (label != -1 and pred == label) or (label == -1 and pred == -1):
                    correct_total += 1
                total_test += 1

    auth_acc = 100 * correct_auth / total_auth if total_auth > 0 else 0
    far = 100 * false_accepts / total_unauth if total_unauth > 0 else 0
    frr = 100 * false_rejects / total_auth if total_auth > 0 else 0
    overall_acc = 100 * correct_total / total_test if total_test > 0 else 0
    avg_confidence = confidence_sum / confidence_count if confidence_count > 0 else 0.0

    val_auth_accuracies.append(auth_acc)
    val_far_rates.append(far)
    val_frr_rates.append(frr)
    val_overall_accuracies.append(overall_acc)

    print(f"[Warm-up Epoch {epoch+1}/{num_epochs_warmup}] "
          f"Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.2f}%, "
          f"Val Overall Acc: {overall_acc:.2f}%, FAR: {far:.2f}%, FRR: {frr:.2f}%, Avg Conf: {avg_confidence:.2f}")

    # salva checkpoint migliore
    if overall_acc > best_overall_acc:
        best_overall_acc = overall_acc
        torch.save(model.state_dict(), "best_model.pth")


# -----------------------------
# Fine-tune completo
# -----------------------------
optimizer = get_optimizers(model, stage="finetune")
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs_finetune)
print("Inizio fine-tuning...")

for epoch in range(num_epochs_finetune):
    model.train()
    running_loss = correct_train = total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        logits, embeddings = model(images, labels)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct_train / total_train)

    # --- VALIDATION OPEN-SET ---
    model.eval()
    correct_auth = total_auth = 0
    false_accepts = false_rejects = 0
    total_unauth = total_test = correct_total = 0
    confidence_sum = confidence_count = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            logits, embeddings = model(images, labels=None)
            probs = F.softmax(logits, dim=1)
            max_probs, predicted = torch.max(probs, 1)

            for pred, label, conf in zip(predicted, labels, max_probs):
                label, pred, conf = label.item(), pred.item(), conf.item()
                confidence_sum += conf
                confidence_count += 1

                if conf < threshold:
                    pred = -1

                if label == -1:
                    total_unauth += 1
                    if pred != -1:
                        false_accepts += 1
                else:
                    total_auth += 1
                    if pred == label:
                        correct_auth += 1
                    else:
                        false_rejects += 1

                if (label != -1 and pred == label) or (label == -1 and pred == -1):
                    correct_total += 1
                total_test += 1

    auth_acc = 100 * correct_auth / total_auth if total_auth > 0 else 0
    far = 100 * false_accepts / total_unauth if total_unauth > 0 else 0
    frr = 100 * false_rejects / total_auth if total_auth > 0 else 0
    overall_acc = 100 * correct_total / total_test if total_test > 0 else 0
    avg_confidence = confidence_sum / confidence_count if confidence_count > 0 else 0.0

    val_auth_accuracies.append(auth_acc)
    val_far_rates.append(far)
    val_frr_rates.append(frr)
    val_overall_accuracies.append(overall_acc)

    print(f"[Fine-tune Epoch {epoch+1}/{num_epochs_finetune}] "
          f"Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.2f}%, "
          f"Val Overall Acc: {overall_acc:.2f}%, FAR: {far:.2f}%, FRR: {frr:.2f}%, Avg Conf: {avg_confidence:.2f}")

    scheduler.step()

    if overall_acc > best_overall_acc:
        best_overall_acc = overall_acc
        torch.save(model.state_dict(), "best_model.pth")


🔧 Inizio warm-up...
[Warm-up Epoch 1/5] Loss: 6.3031, Train Acc: 1.27%, Val Overall Acc: 33.33%, FAR: 0.00%, FRR: 100.00%, Avg Conf: 0.10
[Warm-up Epoch 2/5] Loss: 5.3672, Train Acc: 2.36%, Val Overall Acc: 33.33%, FAR: 0.00%, FRR: 100.00%, Avg Conf: 0.07
[Warm-up Epoch 3/5] Loss: 12.8814, Train Acc: 0.00%, Val Overall Acc: 33.33%, FAR: 0.00%, FRR: 100.00%, Avg Conf: 0.07
[Warm-up Epoch 4/5] Loss: 12.6907, Train Acc: 0.00%, Val Overall Acc: 33.33%, FAR: 0.00%, FRR: 100.00%, Avg Conf: 0.09
[Warm-up Epoch 5/5] Loss: 12.5400, Train Acc: 0.00%, Val Overall Acc: 33.33%, FAR: 0.00%, FRR: 100.00%, Avg Conf: 0.13
🔧 Inizio fine-tuning...
[Fine-tune Epoch 1/30] Loss: 11.8936, Train Acc: 0.00%, Val Overall Acc: 34.67%, FAR: 0.00%, FRR: 98.00%, Avg Conf: 0.16
[Fine-tune Epoch 2/30] Loss: 11.0737, Train Acc: 0.00%, Val Overall Acc: 32.00%, FAR: 14.00%, FRR: 95.00%, Avg Conf: 0.26
[Fine-tune Epoch 3/30] Loss: 10.3729, Train Acc: 0.00%, Val Overall Acc: 34.67%, FAR: 14.00%, FRR: 91.00%, Avg Conf: 0.3